In [ ]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_exportacion_pdf_seriemapas

In [ ]:

fecha_actual = dt.now()
fecha_directorio = str(fecha_actual.strftime("%Y%m%d"))

RUTA_CONSOLIDACION_SALDOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Tramites_Conservacion\2_2_10_Seguimiento_EConservacion\zInsumos\20240903.csv"
FC_TERRENOS_UNIFICADOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"

# TODO: Generación XLSX Resultados
DIRECTORIO_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Tramites_Conservacion\2_2_10_Seguimiento_EConservacion\zReportes"
NOMBRE_XLSX = fecha_directorio+'_Seguimiento_indicador_conservacion.xlsx'

# TODO: Generación Salidas Gráficas
DIRECTORIO_PDF = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Tramites_Conservacion\2_2_10_Seguimiento_EConservacion\Salidas_Graficas"

# TODO: Salida de Estadísticos de Barra
DIRECTORIO_GRAFICO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Tramites_Conservacion\2_2_10_Seguimiento_EConservacion\Estadisticos"

estandarizacion_saldos_conservacion = ['MUNICIPIO',
    'TIPO',
    'RADICADO IGAC',
    'FECHA RAD IGAC',
    'RADICADO BCGS',
    'REFERENCIA CATASTRAL',
    'MUTACIÓN',
    'FECHA DE VISITA',
    'INFORME DE VISITA',
    'SHF',
    'INFORME TÉCNICO UT',
    'ESTADO DE LA REVISIÓN',
    'FECHA REVISIÓN CONSERVACIÓN',
    'ESTADO DE REVISIÓN',
    'ENVIADO A IGAC',
    'FECHA DE ENVÍO'
    ]

estandarizacion_terrenos_saldosConservacion = [
    'codigo',
    'MUNICIPIO',
    'MUTACIÓN',
    'INFORME DE VISITA',
    'SHF',
    'INFORME TÉCNICO UT',
    'ESTADO DE REVISIÓN',
    'ENVIADO A IGAC',
    'SHAPE'
    ]

In [ ]:
df_saldos_conservacion = pd.read_csv(RUTA_CONSOLIDACION_SALDOS)

In [ ]:

# TODO: Se parametriza el .csv de Conservación
df_saldos_conservacion = df_saldos_conservacion[estandarizacion_saldos_conservacion]

# TODO: Se eliminan los datos en blanco, si existen
df_saldos_conservacion = df_saldos_conservacion.dropna(subset=['MUNICIPIO'])

# TODO: Se realiza parametrización de tramites realizados por DT y Ventanilla
df_saldos_conservacion_dt = df_saldos_conservacion[df_saldos_conservacion['TIPO']=='SALDO']
df_saldos_conservacion_ventanilla = df_saldos_conservacion[df_saldos_conservacion['TIPO']=='VENTANILLA']

# TODO: Agrupamiento por tipo de Saldo
df_saldos_conservacion_dt = ((pd.DataFrame(df_saldos_conservacion_dt.groupby(['MUNICIPIO','MUTACIÓN'])['MUTACIÓN'].count())).rename(columns={'MUTACIÓN':'total_saldos'})).reset_index()
df_saldos_conservacion_ventanilla = ((pd.DataFrame(df_saldos_conservacion_ventanilla.groupby(['MUNICIPIO','MUTACIÓN'])['MUTACIÓN'].count())).rename(columns={'MUTACIÓN':'total_saldos'})).reset_index()

# TODO: Pivoteo
df_saldos_conservacion_dt_pivoteo = df_saldos_conservacion_dt.pivot(index='MUNICIPIO', columns='MUTACIÓN', values='total_saldos')
df_saldos_conservacion_ventanilla_pivoteo = df_saldos_conservacion_ventanilla.pivot(index='MUNICIPIO', columns='MUTACIÓN', values='total_saldos')

# TODO: Cambio de nan por 0
df_saldos_conservacion_dt_pivoteo = (df_saldos_conservacion_dt_pivoteo.fillna(0)).astype(int)
df_saldos_conservacion_ventanilla_pivoteo = (df_saldos_conservacion_ventanilla_pivoteo.fillna(0)).astype(int)

with pd.ExcelWriter(os.path.join(DIRECTORIO_XLSX,NOMBRE_XLSX), engine='openpyxl', if_sheet_exists='replace') as writer:
    df_saldos_conservacion_dt_pivoteo.to_excel(writer, sheet_name='segIndDt', index=True)
    df_saldos_conservacion_ventanilla_pivoteo.to_excel(writer, sheet_name='segIndVentanilla', index=True)

In [ ]:
df_terrenos = pd.DataFrame.spatial.from_featureclass(FC_TERRENOS_UNIFICADOS)

estandarizacion_terrenos  = ['codigo','area_ha_cmt12','SHAPE']

In [ ]:
df_terrenos_saldosConservacion = pd.merge(left=df_terrenos,
    right=df_saldos_conservacion,
    left_on='codigo',
    right_on='REFERENCIA CATASTRAL',
    how='inner')

df_terrenos_saldosConservacion = df_terrenos_saldosConservacion[estandarizacion_terrenos_saldosConservacion]

df_terrenos_saldosConservacion

<h3>Generación Capa Geográfica</h3>

In [ ]:
RUTA_BDLOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Tramites_Conservacion\2_2_10_Seguimiento_EConservacion\Base_Datos\2_2_10_SegEConservacion.gdb"
NOMBRE_FEATURE = 'Seguimiento_EConservacion'

df_terrenos_saldosConservacion.spatial.to_featureclass(location=os.path.join(RUTA_BDLOCAL, NOMBRE_FEATURE))
print(f"Se genera el featureclass {NOMBRE_FEATURE}")

<h3>Generación de Salida Gráfica</h3>

In [ ]:

# TODO: Generación Salida Gráfica
funcion_exportacion_pdf_seriemapas.exportacion_salida_grafica('Seguimiento E. Conservacion', DIRECTORIO_PDF)
print(f"Se exporta Salida Gráfica")

<h3>Informes de Visita</h3>

In [ ]:

# TODO: Se filtran aquellos registros que tienen SI en Informe de Vista
df_saldos_conservacion_infVisita = df_saldos_conservacion[df_saldos_conservacion['INFORME DE VISITA']=='SI']

# TODO: Se realiza el conteo
df_saldos_conservacion_infVisita = ((pd.DataFrame(df_saldos_conservacion_infVisita.groupby(['MUNICIPIO','INFORME DE VISITA'])['INFORME DE VISITA'].count())).rename(columns={'INFORME DE VISITA':'total_informes'})).reset_index()

# ** Aplicar tema a la salida gráfica y tipo de texto y tamaño del texto
sns.set_theme(style='whitegrid', font='Tahoma', font_scale=1)

# ** Configuración ejes horizontales en Matplotlib
plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = 'gray'
plt.rcParams['axes.linewidth'] = 0.8

paleta_colores = {
    'BARANOA ': '#4285f4',  # Azul
    'MARIA LA BAJA': '#ea4335', # Naranja
    'REPELON':'#fbbc04',
    'SABANAGRANDE':'#34a853'
}

f, ax = plt.subplots(figsize=(8, 8))

sns.barplot(x='MUNICIPIO', y='total_informes', data=df_saldos_conservacion_infVisita, ax=ax, palette=paleta_colores)

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', fontsize=10, color='black')

# * Ajustar las etiquetas y el título
ax.set_title(f'Informes de Visitas')
ax.set_xlabel('Municipio')
ax.set_ylabel('Total de Informes de Visitas')

NOMBRE_GRAFICO = fecha_directorio + 'Informe_Visitas.png'
ruta_grafico = os.path.join(DIRECTORIO_GRAFICO, NOMBRE_GRAFICO)

plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')

<h3>Trámites Entregados Territoriales</h3>

In [ ]:
df_saldos_conservacion

In [ ]:

# TODO: Se filtran aquellos registros que tienen SI en Enviado IGAC
df_saldos_conservacion_infEnviados = df_saldos_conservacion[df_saldos_conservacion['ENVIADO A IGAC']=='SI']

# TODO: Se realiza el conteo
df_saldos_conservacion_infEnviados = ((pd.DataFrame(df_saldos_conservacion_infEnviados.groupby(['MUNICIPIO','ENVIADO A IGAC'])['ENVIADO A IGAC'].count())).rename(columns={'ENVIADO A IGAC':'total_informes'})).reset_index()

# ** Aplicar tema a la salida gráfica y tipo de texto y tamaño del texto
sns.set_theme(style='whitegrid', font='Tahoma', font_scale=1)

# ** Configuración ejes horizontales en Matplotlib
plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = 'gray'
plt.rcParams['axes.linewidth'] = 0.8

paleta_colores = {
    'BARANOA ': '#4285f4',  # Azul
    'MARIA LA BAJA': '#ea4335', # Naranja
    'REPELON':'#fbbc04',
    'SABANAGRANDE':'#34a853'
}

f, ax = plt.subplots(figsize=(8, 8))

sns.barplot(x='MUNICIPIO', y='total_informes', data=df_saldos_conservacion_infEnviados, ax=ax, palette=paleta_colores)

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', fontsize=10, color='black')

# * Ajustar las etiquetas y el título
ax.set_title(f'Informes Enviados IGAC')
ax.set_xlabel('Municipio')
ax.set_ylabel('Total de Informes Enviados IGAC')

NOMBRE_GRAFICO = fecha_directorio + 'Informe_Enviados.png'
ruta_grafico = os.path.join(DIRECTORIO_GRAFICO, NOMBRE_GRAFICO)

plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')